# RAG with Databricks Vector Search with Context from Multiple Sources

Author: [@MariaKhalusova](https://x.com/mariaKhalusova)

Last updated: Feb 6th, 2025


This notebook complements [this blog post](https://unstructured.io/blog/rag-seamlessly-integrating-context-from-multiple-sources-into-delta-tables-in-databricks) and illustrates how to build RAG over data that was ingested from Amazon S3 bucket and Google Drive into Databricks Delta Table with the Unstructured Platform.

For the details on how to ingest data from multiple sources, preprocess it with Unstructured Platform, write the results into a Delta Table in Databricks, and how to create Databricks Vector Search Index over a Delta Table, please refer to the blog post.

The notebook only covers the RAG setup.

## Prerequisites

* [Sign for the Unstructured For Developers access](https://unstructured.io/developers). Once you do, you can log into the Platform and process up to 1000 pages per day for free for the first 14 days.

* [Sign up for Databricks trial](https://login.databricks.com/?dbx_source=www&intent=SIGN_UP&rl_aid=3709974d-322e-48ff-956b-91c816014d75&tuuid=88100e00-3456-4d28-b4c0-9d3ba1e51166)

* Follow the steps in [the blog post](https://unstructured.io/blog/rag-seamlessly-integrating-context-from-multiple-sources-into-delta-tables-in-databricks) to preprocess the data, set up the Databricks Vector Seach, and obtain necessary authentication details.

* Obtain an OpenAI key

Install the necessary libraries:
* `langchain-openai` & `databricks-langchain` to set up a retriever with the Databricks Vector Search
* `openai` to use the LLMs and embedding models from OpenAI


In [ ]:
!pip install -qU langchain-openai databricks-langchain openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/28.3 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.4/647.4 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/2

Set your environment variables:

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "" # Your OpenAI API key
os.environ["DATABRICKS_HOST"] = "" # Host is your workspace's URL: <https://<workspace-id>.cloud.databricks.com
os.environ["DATABRICKS_TOKEN"] = "" # Your Personal Access Token

## Create a LangChain retriever for your Databricks Vector Search

To embed the user query, use the same embedding model that was used to generate embedding vectors stored in the vector search index. In this case, it's `text-embedding-3-small` from OpenAI.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

Create a vector store.

If you followed the blog post to create the vector search index, you should have the `endpoint_name` and `index_name` ready to use here.

Note, by default, similarity search only returns the primary key and text column. If you want to retrieve the custom metadata associated with the document, pass the additional columns in the columns parameter when initializing the vector store. In this example, we'll use the element `type` metadata to know whether it's a table or not, and `text_as_html` metadata to leverage the preserved table structure.

In [ ]:
from databricks_langchain import DatabricksVectorSearch

index_name = "demo_workspace.default.demo_index"  # Format: "<catalog>.<schema>.<index-name>"
endpoint_name = "uns_demo_vector_search"

vector_store = DatabricksVectorSearch(
    endpoint=endpoint_name,
    index_name=index_name,
    embedding=embeddings,
    text_column="text", # The column name in the index that contains the text data
    columns=["text_as_html", "type"], # metadata columns to retrieve
)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


Set up this vector store as a retriever:

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

Let's try it out.

In [ ]:
query = "Who is Brian W. Nichols?"

In [ ]:
result = retriever.invoke(query)

The first retrieved document is actually a table, and we can display it. You can easily see that the second row contains the exact information that answers the question.

In [ ]:
from IPython.core.display import display, HTML

display(HTML(result[0].metadata["text_as_html"]))

W. Rodney McMullen,63,"Mr. McMullen was elected Chairman of the Board effective January 1, 2015, and Chief Executive Officer effective January 1, 2014. Prior to that, he served as President and Chief Operating Officer from August 2009 to December 2013. Prior to that he held numerous leadership roles, including Vice Chairman, Executive Vice President of Strategy, Planning and Finance, Executive Vice President and Chief Financial Officer, Senior Vice President, Group Vice President and Chief Financial Officer, Vice President, Control and Financial Services, and Vice President, Planning and Capital Management. Mr. McMullen joined Kroger in 1978 as a part-time stock clerk."
Brian W. Nichols,51,"Mr. Nichols was elected Vice President, Corporate Controller in March 2024 and is responsible for oversight of Kroger’s Corporate Accounting and Corporate Tax departments, as well as the Company’s Accounting Centers and Accounting Modernization, Pension Investment, and Insurance and Claims teams. Prior to that, he served as Vice President, Assistant Corporate Controller from April 2021 to March 2024. From May 2018 to April 2021, Mr. Nichols served as Senior Director and Assistant Corporate Controller. Prior to that, he held several leadership roles, including Senior Manager of Corporate and External Financial Reporting and Senior Financial Analyst of SEC Reporting. Mr. Nichols joined Kroger in 2000 as Assistant Controller of the Central Division."


Now, in this example it isn't extremely important to preserve the table structure, but in more dense tables with lots of numerical data, structure can be much more crucial.

Let's build a RAG application in which when we retrieve a table, we'll actually give the LLM the html representation of said table instead of plain text.

First, we'll create a function that generates an answer given a question and retrieved documents:

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_answer(question: str, documents: str):

    prompt = """
You are an assistant that can answer user questions given provided context.
Your answer should be thorough and technical.
If you don't know the answer, or no documents are provided, say 'I do not have enough context to answer the question.'
"""

    augmented_prompt = (
        f"{prompt}"
        f"User question: {question}\n\n"
        f"{documents}"
    )
    response = client.chat.completions.create(
        messages=[
            {'role': 'system', 'content': 'You answer users questions.'},
            {'role': 'user', 'content': augmented_prompt},
        ],
        model="gpt-4o-2024-11-20",
        temperature=0,
    )

    return response.choices[0].message.content


Next, we'll create a helper function that will format the retrieved documents in the following way: if we retrieved a table, then use the `text_as_html` representation of the table as a source, otherwise, just use the text:

In [ ]:
def format_docs(docs):
  useful_content = [doc.page_content if doc.metadata["type"] != "Table" else doc.metadata["text_as_html"] for doc in docs]

  return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc
                for i, doc in enumerate(useful_content)
            ]
        )

Bring everything together:
* Given query, invoke the retriever and get the documents
* Format the documents to preserve the table structure
* Pass the formatted documents and the user query to the LLM to generate an answer

In [ ]:
def rag(query):
  docs = retriever.invoke(query)
  documents = format_docs(docs)
  answer = generate_answer(query, documents)
  return answer

In [ ]:
query = "What is the exact Kroger's operating profit in 2022?"

rag(query)

"Based on the provided documents, Kroger's operating profit for the fiscal year 2022 (52 weeks) was **$4.126 billion**. This figure is explicitly stated in the financial table in Document 2."

Let's see that financial table from the Document 2 to confirm the answer:

In [ ]:
results = retriever.invoke(query)

In [ ]:
from IPython.core.display import display, HTML

display(HTML(results[2].metadata["text_as_html"]))